# Sparse Indexes in DynamoDB

Keywords: AWS, Amazon, DynamoDB

所谓 Spare Indexes 就是当 Index 里的 PK 或者 SK 在 Base Table 的 Item 中不存在时, 这个 Item 就不会在 Index 中出现. 这适合你只需要把一小部分的 Item 放到 Index 中的情况.

官方文档中有个很好的例子. 电商有一个表记录了所有的 Order. PK 是 ``CustomerId``, SK 是 ``OrderId``. 用户登录后需要查看已经下单, 但还没有送达的 Order. 偶尔需要查看所有的 Order 的历史记录. 显然前者是更高频的需求. 考虑到一个 Customer 一辈子下的单不可能太夸张, 所以你完全可以用 ``CustomerId`` 获得所有 Order, 然后再内存中进行排序. 但考虑到一般顶多有 1 到 2 个订单处于这种情况, 为了这 2 个订单查询了几十个订单还是比较浪费. 

这里介绍一个利用 Sparse Indexes 对齐进行优化的办法. 你可能有一个 Attribute 叫 ``CreateAt``, 记录了订单创建的时间, 还有一个 Attribute 叫做 ``Status``, 它的值可能是 ``pending``, ``delivering``, ``delivered``. 这里你不要直接用 ``Status`` 做 index, 而是专门创建一个 attribuge 叫做 ``OrderCreateAt``, 它是一个时间戳, 只有这个订单处于 ``pending``, ``delivering`` 的状态时它有值. 处于 ``delivered`` 的状态时候这个 attribute 就没有了. 然后你可以创建一个 Index, PK 是 CustomerId, SK 是 ``OrderCreateAt``. 这样你用这个 index 可以轻松找到谋个用户还没有送达的订单, 并按照时间顺序排列.

**Reference**:

- https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/bp-indexes-general-sparse-indexes.html

In [1]:
import pynamodb_mate as pm
from boto_session_manager import BotoSesManager

bsm = BotoSesManager(profile_name="bmt_app_dev_us_east_1")
with bsm.awscli():
    pm.Connection()

## Example 1

下面这个例子说明了 sort key 不能是 Null.

In [2]:
class Model1(pm.Model):
    class Meta:
        table_name = "sparce_indexes_test_1"
        region = "us-east-1"
        billing_mode = pm.PAY_PER_REQUEST_BILLING_MODE

    pk = pm.UnicodeAttribute(hash_key=True)
    sk = pm.UnicodeAttribute(range_key=True)


Model1.create_table(wait=True)

# Not gonna work
model = Model1(doc_id="pk-1", sk=None)
model.save()

ValueError: Attribute doc_id specified does not exist

## Example 2

下面这个例子说明了如果你的 Index 只有 PK, 且 PK 这个 attribute 在 Base table 中的 Item 中不存在, 那么这个 Item 就不会出现在 Index 里.

In [5]:
class Model2Index(pm.GlobalSecondaryIndex):
    class Meta:
        index = "model2-index"
        projection = pm.KeysOnlyProjection()

    gsi_pk = pm.UnicodeAttribute(hash_key=True)


class Model2(pm.Model):
    class Meta:
        table_name = "sparce_indexes_test_2"
        region = "us-east-1"
        billing_mode = pm.PAY_PER_REQUEST_BILLING_MODE

    pk = pm.UnicodeAttribute(hash_key=True)
    gsi_pk = pm.UnicodeAttribute(null=True)

    index = Model2Index()

Model2.create_table(wait=True)

In [6]:
Model2(pk="id-1", gsi_pk=None).save()
Model2(pk="id-2", gsi_pk="id-2-gsi-pk").save()

{'ConsumedCapacity': {'CapacityUnits': 2.0,
  'TableName': 'sparce_indexes_test_2'}}

In [7]:
for i in Model2.index.query(hash_key=None):
    print(i)

QueryError: Failed to query items: An error occurred (ValidationException) on request (I1NDF93NG76SP0Q6RC410310FBVV4KQNSO5AEMVJF66Q9ASUAAJG) on table (sparce_indexes_test_2) when calling the Query operation: ExpressionAttributeValues contains invalid value: Supplied AttributeValue is empty, must contain exactly one of the supported datatypes for key :0

In [8]:
for i in Model2.index.query(hash_key="id-2-gsi-pk"):
    print(i.attribute_values)

{'gsi_pk': 'id-2-gsi-pk', 'pk': 'id-2'}


## Example 3

下面这个例子说明了如果你的 Index 有 PK 和 SK, 且 PK 和 SK 的 attribute 中的任意一个在 Base table 中的 Item 中不存在, 那么这个 Item 就不会出现在 Index 里.

In [ ]:
class Model3Index(pm.GlobalSecondaryIndex):
    class Meta:
        index = "model3-index"
        projection = pm.KeysOnlyProjection()

    gsi_pk = pm.UnicodeAttribute(hash_key=True)
    gsi_sk = pm.UnicodeAttribute(range_key=True)


class Model3(pm.Model):
    class Meta:
        table_name = "sparce_indexes_test_3"
        region = "us-east-1"
        billing_mode = pm.PAY_PER_REQUEST_BILLING_MODE

    pk = pm.UnicodeAttribute(hash_key=True)
    gsi_pk = pm.UnicodeAttribute(null=True)
    gsi_sk = pm.UnicodeAttribute(null=True)

    index = Model3Index()

Model3.create_table(wait=True)

In [ ]:
Model3(pk="id-1", gsi_pk=None, gsi_sk=None).save()
Model3(pk="id-2", gsi_pk="id-2-gsi-pk", gsi_sk=None).save()
Model3(pk="id-3", gsi_pk=None, gsi_sk="id-3-gsi-sk").save()
Model3(pk="id-4", gsi_pk="id-4-gsi-pk", gsi_sk="id-4-gsi-sk").save() # only this will be in the index

In [ ]:
# Nothing there
for i in Model3.index.query(hash_key="id-2-gsi-pk"):
    print(i.attribute_values)

In [ ]:
for i in Model3.index.query(hash_key="id-4-gsi-pk"):
    print(i.attribute_values)

## Example 4

这个例子是说明在 GSI 中, PK 和 SK 合起来并不需要是唯一的. GSI 只是像 DynamoDB Table, 但它不是一个真正的 GSI Table.

In [ ]:
class Model4Index(pm.GlobalSecondaryIndex):
    class Meta:
        index = "model4-index"
        projection = pm.KeysOnlyProjection()

    gsi_pk = pm.UnicodeAttribute(hash_key=True)
    gsi_sk = pm.UnicodeAttribute(range_key=True)


class Model4(pm.Model):
    class Meta:
        table_name = "sparce_indexes_test_4"
        region = "us-east-1"
        billing_mode = pm.PAY_PER_REQUEST_BILLING_MODE

    pk = pm.UnicodeAttribute(hash_key=True)
    gsi_pk = pm.UnicodeAttribute(null=True)
    gsi_sk = pm.UnicodeAttribute(null=True)

    index = Model4Index()

Model4.create_table(wait=True)

In [ ]:
Model4(pk="id-1", gsi_pk="gsi-pk", gsi_sk="gsi-sk").save()
Model4(pk="id-2", gsi_pk="gsi-pk", gsi_sk="gsi-sk").save()

In [ ]:
for i in Model4.index.query(hash_key="gsi-pk"):
    print(i.attribute_values)